# Revisiting the Strategies in Federated Learning

This notebook contains the solution to the two exercises proposed in the practical Unit 5 of the Machine Learning II course. 

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from typing import List, Dict, Optional, Tuple, Union
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
import flwr as fl
import numpy as np
from flwr.common import EvaluateRes, FitRes, Scalar
from flwr.common import Metrics
from flwr.server.client_proxy import ClientProxy

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def split_index(a, n):
    s = np.array_split(np.arange(len(a)), n)
    return s

def load_datasets(num_clients: int):
    # Distribute it to train and test set
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    # Normalize data
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0

    x_train, y_train = x_train[:10_000], y_train[:10_000]
    x_test, y_test = x_test[:1000], y_test[:1000]

    # Randomize the datasets
    x_train, y_train = unison_shuffled_copies(x_train, y_train)
    x_test, y_test = unison_shuffled_copies(x_test, y_test)

    # Split training set into 'num_clients' partitions to simulate the individual dataset
    train_index = split_index(x_train, num_clients)
    test_index = split_index(x_test, num_clients)

    # Split each partition
    train_ds = []
    val_ds = []
    test_ds = []
    for cid in range(num_clients):
        val_size = len(train_index[cid]) // 10
        train_input_data, train_output_data = x_train[train_index[cid]], y_train[train_index[cid]]
        val_input_data, val_output_data = train_input_data[:val_size], train_output_data[:val_size]
        train_input_data, train_output_data = train_input_data[val_size:], train_output_data[val_size:]
        train_dataset = (train_input_data, train_output_data)
        val_dataset = (val_input_data, val_output_data)
        test_dataset = (x_test[test_index[cid]], y_test[test_index[cid]])
        train_ds.append(train_dataset)
        val_ds.append(val_dataset)
        test_ds.append(test_dataset)
    
    return train_ds, val_ds, test_ds

def get_parameters(net) -> List[np.array]:
    return net.get_weights()

def set_parameters(net, parameters: List[np.ndarray]):
    net.set_weights(parameters)
    return net

def train(net, trainloader, batch_size: int, epochs: int):
    net.fit(trainloader[0], trainloader[1],
            epochs=epochs, batch_size=batch_size, steps_per_epoch=1)
    return net

def test(net, testloader):
    loss, accuracy = net.evaluate(testloader[0], testloader[1])
    return loss, accuracy

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        self.net = set_parameters(self.net, parameters)
        self.net = train(self.net, self.trainloader, batch_size=config['batch_size'], epochs=config['epochs'])
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        self.net = set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        print(f"[Client {self.cid}] loss:{loss}, Client {self.cid} accuracy:{accuracy}")
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}
    

class AggregateCustomMetricStrategy(fl.server.strategy.FedAvg):
    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation accuracy using weighted average."""

        if not results:
            return None, {}

        aggregated_loss, aggregated_metrics = super().aggregate_evaluate(server_round, results, failures)

        accuracies = [r.metrics["accuracy"] * r.num_examples for _, r in results]
        examples = [r.num_examples for _, r in results]

        aggregated_accuracy = sum(accuracies) / sum(examples)
        print(f"Round {server_round} accuracy aggregated from client results: {aggregated_accuracy}")

        return aggregated_loss, {"accuracy": aggregated_accuracy}


2024-03-13 20:06:27.898594: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-13 20:06:27.899022: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-13 20:06:27.901854: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-13 20:06:27.939568: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 20:06:28.606542: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
NUM_CLIENTS = 100
NUM_ROUNDS = 100
MODEL_IDEN = 1

   
def generate_ann(iden: int = MODEL_IDEN, lr: float = 1e-4):
    if iden == 0:
        model = tf.keras.Sequential([
            Flatten(input_shape=(32, 32, 3)),
            Dense(64, activation='relu'),
            Dense(64, activation='relu'),
            Dense(10, activation='softmax')
        ])
    elif iden == 1:
        model = tf.keras.Sequential([
            Conv2D(32, 5, activation='relu', input_shape=(32, 32, 3)),
            Conv2D(32, 5, activation='relu', input_shape=(32, 32, 3)),
            BatchNormalization(1),
            Conv2D(128, 5, activation='relu'),
            Dropout(0.2),
            Flatten(),
            Dense(30, activation='relu'),
            Dense(10, activation='softmax')
        ])
    model.compile(
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(lr),
        metrics=['accuracy']
    )
    return model
    
    

model = generate_ann()
params = get_parameters(model)
print(model.summary())
del model 

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)


def evaluate_fn(
        server_round: int, 
        parameters: fl.common.NDArrays, 
        config: Dict[str, fl.common.Scalar]
    ) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    dataset = testloader[0]
    set_parameters(model, parameters)  # Update model with the latest parameters
    loss, accuracy = test(model, dataset)
    print(f"Server-side evaluation round {server_round} with loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

def client_fn(cid) -> FlowerClient:
    net = generate_ann()
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

def fit_config(server_round: int):
    return dict(server_round=server_round, epochs=10, batch_size=100)

strategy = AggregateCustomMetricStrategy(
    fraction_fit=1, 
    fraction_evaluate=0.2, 
    min_fit_clients=20,
    min_evaluate_clients=40,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
    on_fit_config_fn=fit_config,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),  # Just three rounds
    strategy=strategy,
)

/home/ana/Documents/ml2-labs/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
2024-03-13 20:06:29.216664: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 32)     │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 24, 24, 32)     │            96 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 20, 20, 128)    │       102,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 51200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 30)             │     1,536,030 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           310 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,667,028 (6.36 MB)

 Trainable params: 1,666,980 (6.36 MB)

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 [repeated 68x across cluster]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.4444 - loss: 1.4117 [repeated 2x across cluster]
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.4111 - loss: 1.6608
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.4111 - loss: 1.7270
(DefaultActor pid=483441) [Client 94] fit, config: {'server_round': 14, 'epochs': 10, 'batch_size': 100} [repeated 14x across cluster]
(DefaultActor pid=483414) Epoch 9/10 [repeated 129x across cluster]


(DefaultActor pid=483419) 2024-03-13 20:17:32.065193: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence [repeated 57x across cluster]
(DefaultActor pid=483419) 	 [[{{node IteratorGetNext}}]] [repeated 57x across cluster]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step - accuracy: 0.5444 - loss: 1.2192 [repeated 57x across cluster]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 [repeated 60x across cluster]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step - accuracy: 0.5889 - loss: 1.3264
(DefaultActor pid=483413) [Client 84] fit, config: {'server_round': 14, 'epochs': 10, 'batch_size': 100} [repeated 13x across cluster]
(DefaultActor pid=483414) Epoch 3/10 [repeated 128x across cluster]


(DefaultActor pid=483418) 2024-03-13 20:17:37.077881: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence [repeated 72x across cluster]
(DefaultActor pid=483418) 	 [[{{node IteratorGetNext}}]] [repeated 72x across cluster]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step - accuracy: 0.6000 - loss: 1.2471 [repeated 70x across cluster]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 [repeated 78x across cluster]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step - accuracy: 0.6889 - loss: 1.1145 [repeated 2x across cluster]
